In [ ]:
import math as m
import numpy as np
import sympy as sym
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
from tqdm import tqdm
import torch
from itertools import product
import cmath as cm

In [ ]:
# defininf the states
# defininsg coefficeients sybols

def Creating_states(coeff, abstract = False):    # coeff list like [a0,a1,b0,b1]
    if abstract == True:
        a0 = sym.symbols('a0')
        a1 = sym.symbols('a1')
        b0 = sym.symbols('b0')
        b1 = sym.symbols('b1')
    else:
        a0 = coeff[0]
        a1 = coeff[1]
        b0 = coeff[2]
        b1 = coeff[3]
    psi0 = [a0,a1]      # defining states
    psi1 = [b0,b1]
    return([psi0,psi1])

In [ ]:
def polar(z):
    real_part = z.real
    imaginary_part = z.imag
    r = np.sqrt(real_part**2 + imaginary_part**2)
    theta = np.arctan(imaginary_part/real_part)
    return r,theta

In [ ]:
def roundc(c, digits):
    if c.imag == 0:
        return round(c.real, digits)
    else:
        return round(c.real, digits) + round(c.imag, digits) * 1j

In [ ]:
def update_lr(lr, i):
    return lr/(i+1)

In [ ]:
# creating the SIC POVM matrices
w = m.e**((2/3)*m.pi*(1j))
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # an array of POVM direction vectors
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]   # a list of POVM matrix

M = [[np.trace(np.dot(POVM_elts[i],POVM_elts[j])) for i in range(len(POVM_elts))] for j in range(len(POVM_elts))]     # creating M matrix using POVM definition

u_0 = [1/3 for i in range(9)]           # cerating u_0 vector, to create the inverse matrix
M_inv = 3*np.outer(u_0,u_0) + 12*(np.eye(9) - np.outer(u_0,u_0))        # creating the inverse matrix

In [ ]:
def fnj_vec(N, coeff, priors, POVM_elts):

    initial_states = Creating_states(coeff = coeff, abstract=False)     # Creating the two states with these coefficients
    psi0 = initial_states[0]
    psi1 = initial_states[1]    # created the states to be discriminated
    
    psi0sq = []
    psi1sq = []
    [[psi0sq.append(i*j) for i in psi0] for j in psi0]
    [[psi1sq.append(i*j) for i in psi1] for j in psi1]   
    psi0psi0 = [psi0sq[0], np.sqrt(psi0sq[1]**2+psi0sq[2]**2), psi0sq[3]]    
    psi1psi1 = [psi1sq[0], np.sqrt(psi1sq[1]**(2)+psi1sq[2]**(2)), psi1sq[3]]    # creating square states
    
    vec_psi0psi0 = np.array(psi0psi0)     
    vec_psi1psi1 = np.array(psi1psi1)
    rho = priors[0]*np.outer(vec_psi0psi0, vec_psi0psi0)+ priors[1]*np.outer(vec_psi1psi1, vec_psi1psi1)     # theoretical density matrix with priors 1/2 each.

    prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] 
    prob_vec = [i.real for i in prob_vec if abs(i.imag) < .01]          # cleaned up theoretical prob vector

    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]

    return nj_vec

In [ ]:
#Linear Inversion

def solve_quadratic(a, b, c):

    if a == 0 and c == 0:
        sol1 = 0
        sol2 = 0

    dis = b * b - 4 * a * c 
    sqrt_val = m.sqrt(abs(dis))

    if dis > 0: 
        sol1 = (-b - sqrt_val)/(2 * a)
        sol2 = (-b + sqrt_val)/(2 * a)
     
    elif dis == 0: 
        sol1 = -b / (2 * a)
        sol2 = -b / (2 * a)
     
    # when discriminant is less than 0
    else:
        sol1 = - b / (2 * a) - 1j*sqrt_val/ (2 * a)
        sol2 = - b / (2 * a) + 1j*sqrt_val/ (2 * a)
        
    return sol1, sol2
    
    '''coefficients = np.array([a, b, c])
    roots = np.roots(coefficients)
    return roots'''

def experiment_fid(N, coeff, priors, POVM_elts, M_inv):
    initial_states = Creating_states(coeff = coeff, abstract=False)     # Creating the two states with these coefficients
    psi0 = initial_states[0]
    psi1 = initial_states[1]    # created the states to be discriminated

    nj_vec = fnj_vec(N, coeff, priors, POVM_elts)
    pj_num_vec = [i/N for i in nj_vec]                                  # numerical prob vector     
    
    r_vec = np.dot(M_inv,pj_num_vec)
    
    # constructing the rho numrical using the r_vector etc.
    rho_num_list = [r_vec[i]*POVM_elts[i] for i in range(len(POVM_elts))]   # list of matrices, see equation 7 in Hillery notes pair_disc.pdf
    rho_num = np.zeros_like(rho_num_list[0])
    for matrix in rho_num_list:
        rho_num = np.add(rho_num, matrix)       # created the numerical rho
     
    eigenvalues, eigenvectors = np.linalg.eig(rho_num)
    evals = np.array([i.real for i in eigenvalues if abs(i.imag)< .001])
    #evals = np.array([abs(i) for i in eigenvalues])
    index = np.argmin(evals)
    eigenvector = eigenvectors[:,index]

    c00=eigenvector[0]
    #c01=eigenvector[1]
    c01t=eigenvector[1]/m.sqrt(2)
    c11=eigenvector[2]

    f0,f1=solve_quadratic(c00, 2*c01t, c11)
    f0p,f1p=solve_quadratic(c11, 2*c01t, c00)
        
    #all possible (positive) coeffs
    a1 = np.conj(f0)/(m.sqrt(1+ (abs(f0))**2)) 
    a2 = 1/(m.sqrt(1+ (abs(f0))**2))
    a3 = np.conj(f0p)/(m.sqrt(1+ (abs(f0p))**2)) 
    a4 = 1/(m.sqrt(1+ (abs(f0p))**2))
    a5 = np.conj(f1)/(m.sqrt(1+ (abs(f1))**2)) 
    a6 = 1/(m.sqrt(1+ (abs(f1))**2))
    a7 = np.conj(f1p)/(m.sqrt(1+ (abs(f1p))**2))
    a8 = 1/(m.sqrt(1+ (abs(f1p))**2))

    #lists of all possible coeffs, including megative and complex conjugates. each sublist correspons to a "physical sector" 
    v12 = [a1,a2,-a1,-a2]
    v34 = [a3,a4,-a3,-a4]
    v56 = [a5,a6,-a5,-a6]
    v78 = [a7,a8,-a7,-a8]

    pairs12 = [[i,j] for i in v12 for j in v12 if i!=j and i!= -j]
    #print(f'len(pairs12): {len(pairs12)}') 
    pairs34 = [[i,j] for i in v34 for j in v34 if i!=j and i!= -j]
    #print(f'len(pairs34): {len(pairs34)}')
    pairs56 = [[i,j] for i in v56 for j in v56 if i!=j and i!= -j] 
    #print(f'len(pairs56): {len(pairs56)}')
    pairs78 = [[i,j] for i in v78 for j in v78 if i!=j and i!= -j]
    #print(f'len(pairs78): {len(pairs78)}')

    pairs = pairs12 + pairs34 + pairs56 + pairs78
    
    #all possible fidelities
    fid_psi0 = [abs(np.vdot(i, psi0))**2 for i in pairs]
    fid_psi1 = [abs(np.vdot(j, psi1))**2 for j in pairs]

    #Create the dictionaries for the fidelities and the corresponding num wavefunctions
    fid_psi0_dict = dict(zip(fid_psi0, pairs))
    fid_psi1_dict = dict(zip(fid_psi1, pairs))
    
    #find the max fidelity and the corresponding num wavefunction
    fid0 = max(fid_psi0_dict.keys())
    psi_num0 = fid_psi0_dict[fid0]
    fid1 = max(fid_psi1_dict.keys())
    psi_num1 = fid_psi1_dict[fid1]


    #extract p0 and p1 numerical from rho_num
    psi0sqn = []
    psi1sqn = []
    [[psi0sqn.append(i*j) for i in psi_num0] for j in psi_num0]
    [[psi1sqn.append(i*j) for i in psi_num1] for j in psi_num1]
    psi0psi0n = [psi0sqn[0], np.sqrt(psi0sqn[1]**2+psi0sqn[2]**2), psi0sqn[3]]    
    psi1psi1n = [psi1sqn[0], np.sqrt(psi1sqn[1]**(2)+psi1sqn[2]**(2)), psi1sqn[3]]    # creating square states
    vec_psi0psi0n = np.array(psi0psi0n)     
    vec_psi1psi1n = np.array(psi1psi1n)

    p0c = (np.vdot(vec_psi0psi0n,rho_num @ vec_psi0psi0n) - (abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n)))**2)/(1-(abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n)))**2)
    #p1 = (np.vdot(vec_psi1psi1n,rho_num @ vec_psi1psi1n) - (abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n)))**2)/(1-(abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n)))**2)
    
    p0 = abs(p0c)

    #print(f'p0: {p0}')
    #print(f'fid0: {fid0}, fid1: {fid1}')
    #print(f'psi_num0: {psi_num0}, psi_num1: {psi_num1}')


    #print(p0, p1, polar(psi_num0[0]), polar(psi_num0[1]), polar(psi_num1[0]), polar(psi_num1[1]))

    a = np.arccos(np.sqrt(p0)) #a = np.arcsin(np.sqrt(p1))
    a0 = cm.polar(psi_num0[0])[0]
    th0 = np.arccos(a0)
    a1 = cm.polar(psi_num1[0])[0]
    th1 = np.arccos(a1)
    ph0 = cm.polar(psi_num0[1])[1] - cm.polar(psi_num0[0])[1]
    ph1 = cm.polar(psi_num1[1])[1] - cm.polar(psi_num1[0])[1]

    #print(f't0: {t0}, t1: {t1}')
    #print(f'th0: {th0}, th1: {th1}')
    #print(f'ph0: {ph0}, ph1: {ph1}')

    '''th0 = th0 + np.pi
    th1 = th1 + np.pi

    prova0 = [np.cos(th0), np.sin(th0)]
    prova1 = [np.cos(th1), np.sin(th1)]
    print(f'fid0p: {abs(np.vdot(prova0, psi0))**2}')
    print(f'fid1p: {abs(np.vdot(prova1, psi1))**2}')'''

    return a, ph0, ph1, th0, th1 

In [ ]:
def torch_out(a, f0, f1, t0, t1):

    y_pred = torch.empty(9)
    
    w0 = (torch.cos(a))**2
    w1 = (torch.sin(a))**2
    a0 = torch.cos(t0)
    b0 = torch.sin(t0)
    a1 = torch.cos(t1)
    b1 = torch.sin(t1)

    n2 = torch.tensor([2.0])
    
    y_pred[0] = (1/6)*(w0*(b0**2)*(1 + a0**2 - 2*torch.sqrt(n2)*torch.cos(f0)*a0*b0) + w1*(b1**2)*(1 + a1**2 - 2*torch.sqrt(n2)*torch.cos(f1)*a1*b1)) #p1
    y_pred[1] = (1/6)*(w0*(a0**2)*(1 + b0**2 - 2*torch.sqrt(n2)*torch.cos(f0)*a0*b0) + w1*(a1**2)*(1 + b1**2 - 2*torch.sqrt(n2)*torch.cos(f1)*a1*b1)) #p2
    y_pred[2] = (1/6)*(w0*(a0**4 + b0**4 - 2*torch.cos(2*f0)*(a0**2)*(b0**2)) + w1*(a1**4 + b1**4 - 2*torch.cos(2*f1)*(a1**2)*(b1**2))) #p3
    y_pred[3] = (1/6)*(w0*(b0**2)*(1 + a0**2 - 2*torch.sqrt(n2)*torch.cos(f0 + 2*torch.pi/3)*a0*b0) + w1*(b1**2)*(1 + a1**2 - 2*torch.sqrt(n2)*torch.cos(f1 + 2*torch.pi/3)*a1*b1)) #p4
    y_pred[4] = (1/6)*(w0*(a0**2)*(1 + b0**2 - 2*torch.sqrt(n2)*torch.cos(f0 - 4*torch.pi/3)*a0*b0) + w1*(a1**2)*(1 + b1**2 - 2*torch.sqrt(n2)*torch.cos(f1 + 2*torch.pi/3)*a1*b1)) #p5
    y_pred[5] = (1/6)*(w0*(a0**4 + b0**4 - 2*torch.cos(2*f0 - 2*torch.pi/3)*(a0**2)*(b0**2)) + w1*(a1**4 + b1**4 - 2*torch.cos(2*f1 - 2*torch.pi/3)*(a1**2)*(b1**2))) #p6
    y_pred[6] = (1/6)*(w0*(b0**2)*(1 + a0**2 - 2*torch.sqrt(n2)*torch.cos(f0 - 2*torch.pi/3)*a0*b0) + w1*(b1**2)*(1 + a1**2 - 2*torch.sqrt(n2)*torch.cos(f1 - 2*torch.pi/3)*a1*b1)) #p7
    y_pred[7] = (1/6)*(w0*(a0**2)*(1 + b0**2 - 2*torch.sqrt(n2)*torch.cos(f0 - 2*torch.pi/3)*a0*b0) + w1*(a1**2)*(1 + b1**2 - 2*torch.sqrt(n2)*torch.cos(f1 - 2*torch.pi/3)*a1*b1)) #p8
    y_pred[8] = (1/6)*(w0*(a0**4 + b0**4 - 2*torch.cos(2*f0 - 4*torch.pi/3)*(a0**2)*(b0**2)) + w1*(a1**4 + b1**4 - 2*torch.cos(2*f1 + 2*torch.pi/3)*(a1**2)*(b1**2))) #p9

    return y_pred

In [ ]:
def floss(ny, a, f0, f1, t0,  t1):
    y_pred = torch_out(a, f0, f1, t0, t1)
    total_ny = torch.sum(ny)
    log_likelyhood = -(1/total_ny)*torch.sum(ny * torch.log(y_pred))
    return log_likelyhood

In [ ]:
# Loss vs alpha

coeff = [0**.5,1**.5,1**.5,0**.5] 
priors = [.5,.5]
N = 1000

nj_vec = fnj_vec(N, coeff, priors, POVM_elts)
    
nj_vec = torch.tensor(nj_vec, dtype = torch.int64)

# Define your fixed values for f0, f1, t0, t1
f0_fixed = torch.tensor([torch.pi / 2])
f1_fixed = torch.tensor([torch.pi / 4])
t0_fixed = torch.tensor([torch.pi / 4])
t1_fixed = torch.tensor([torch.pi / 2])

# Define the range of values for a
a_values = torch.linspace(0, 2 * np.pi, 100)

# Compute torch_out for each value of a
torch_out_values = [torch_out(a,f0_fixed,f1_fixed,t0_fixed,t1_fixed) for a in a_values]
floss_values = [floss(nj_vec, a, f0_fixed, f1_fixed, t0_fixed, t1_fixed) for a in a_values]

# Plot the dependence of torch_out on a
plt.plot(a_values, floss_values)
plt.xlabel('a')
plt.ylabel('floss')
plt.show()

In [ ]:
# Loss vs f0

coeff = [0**.5,1**.5,1**.5,0**.5] 
priors = [.5,.5]
N = 1000

nj_vec = fnj_vec(N, coeff, priors, POVM_elts)
    
nj_vec = torch.tensor(nj_vec, dtype = torch.int64)

# Define your fixed values for a, f1, t0, t1
a_fixed = torch.tensor([torch.pi / 4])
f1_fixed = torch.tensor([torch.pi / 4])
t0_fixed = torch.tensor([torch.pi / 4])
t1_fixed = torch.tensor([torch.pi / 4])

# Define the range of values for f0
f0_values = torch.linspace(0, 2 * np.pi, 100)

# Compute torch_out for each value of f0
torch_out_values = [torch_out(a_fixed, f0, f1_fixed, t0_fixed, t1_fixed) for f0 in f0_values]
floss_values = [floss(nj_vec, a_fixed, f0, f1_fixed, t0_fixed, t1_fixed) for f0 in f0_values]

# Plot the dependence of torch_out on f0
plt.plot(a_values, floss_values)
plt.xlabel('f0')
plt.ylabel('floss')
plt.show()

In [ ]:
# Loss vs t0

coeff = [0**.5,1**.5,1**.5,0**.5] 
priors = [.5,.5]
N = 1000

nj_vec = fnj_vec(N, coeff, priors, POVM_elts)
    
nj_vec = torch.tensor(nj_vec, dtype = torch.int64)

# Define your fixed values for a, f0, f1, t1
a_fixed = torch.tensor([torch.pi / 4])
f0_fixed = torch.tensor([torch.pi / 4])
f1_fixed = torch.tensor([torch.pi / 4])
t1_fixed = torch.tensor([torch.pi / 4])

# Define the range of values for t0
t0_values = torch.linspace(0, 2 * np.pi, 100)

# Compute torch_out for each value of t0
torch_out_values = [torch_out(a_fixed, f0_fixed, f1_fixed, t0, t1_fixed) for t0 in t0_values]
floss_values = [floss(nj_vec, a_fixed, f0_fixed, f1_fixed, t0, t1_fixed) for t0 in t0_values]

# Plot the dependence of torch_out on t0
plt.plot(a_values, floss_values)
plt.xlabel('t0')
plt.ylabel('floss')
plt.show()

In [ ]:
def grad_fid(learning_rate, N, coeff, priors, POVM_elts):

    nj_vec = fnj_vec(N, coeff, priors, POVM_elts)

    ##################################### GRADIENT DESCENT ##############################################
    
    nj_vec = torch.tensor(nj_vec, dtype = torch.int64)

    al, f0l, f1l, t0l, t1l = experiment_fid(5000, coeff, priors, POVM_elts, M_inv)

    a = torch.tensor([al], requires_grad=True)
    f0 = torch.tensor([f0l], requires_grad=True)
    f1 = torch.tensor([f1l], requires_grad=True)
    t0_values = [torch.tensor([t0l], requires_grad=True)]
    t1_values = [torch.tensor([t1l], requires_grad=True)]
    #t0_values = [torch.tensor([t0l], requires_grad=True), torch.tensor([t0l + np.pi], requires_grad=True)]
    #t1_values = [torch.tensor([t1l], requires_grad=True), torch.tensor([t1l + np.pi], requires_grad=True)]

    initial_values = [[a], [f0], [f1], t0_values, t1_values]
    initial_conditions = list(product(*initial_values))



    # Create a list to store the loss at each epoch
    loss_history = []
    a_grad = []
    f0_grad = []
    f1_grad = []
    t0_grad = []
    t1_grad = []

    # Run gradient descent for each initial condition
    solutions = []
    for ip in tqdm(initial_conditions):
        # Set the initial condition
        a, f0, f1, t0, t1 = ip
        # Train for N epochs
        for i in range(100):
            
            # Forward pass:
            loss = floss(nj_vec, a, f0, f1, t0, t1)
            # append the current loss to the history
            loss_history.append(loss.item())
            #print(f'Epoch {i}, Loss {loss.item()}')
            

            # Backward pass: compute gradients
            loss.backward()

            a_grad.append(a.grad.item())
            f0_grad.append(f0.grad.item())
            f1_grad.append(f1.grad.item())
            t0_grad.append(t0.grad.item())
            t1_grad.append(t1.grad.item())

            # Update parameters using gradient descent
            with torch.no_grad():
                a -= learning_rate * a.grad
                f0 -= learning_rate * f0.grad
                f1 -= learning_rate * f1.grad
                t0 -= learning_rate * t0.grad
                t1 -= learning_rate * t1.grad

            # Zero gradients for the next iteration
            a.grad.zero_()
            f0.grad.zero_()
            f1.grad.zero_()
            t0.grad.zero_()
            t1.grad.zero_()

        # Store the final solution and its loss
        solutions.append((initial_conditions[0], loss.item()))
        #print(f'initial conditions = {ip}')
        #print(f'loss = {loss.item()}')

    # Find the solution with the lowest loss
    best_solution, best_loss = min(solutions, key=lambda solution: solution[1])

    a, f0, f1, t0, t1 = best_solution
    #print(f'best_solution = {best_solution}')
    #print(f'best_loss = {best_loss}')

    # after the training loop
    plt.figure()
    plt.plot(loss_history, label='Loss')
    plt.plot(a_grad, label='a_grad')
    plt.plot(f0_grad, label='f0_grad')
    plt.plot(f1_grad, label='f1_grad')
    plt.plot(t0_grad, label='t0_grad')
    plt.plot(t1_grad, label='t1_grad')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()                                  
    
    p0 = (torch.cos(a))**2
    p1 = (torch.sin(a))**2
    a0 = torch.cos(t0)
    a1 = torch.sin(t0)
    b0 = torch.cos(t1)
    b1 = torch.sin(t1)
    ph0 = torch.exp(1j*f0) #torch.cos(f0) + 1j*torch.sin(f0)
    ph1 = torch.exp(1j*f1) #torch.cos(f1) + 1j*torch.sin(f1)
    
    psi0n = torch.tensor([a0, ph0*a1], dtype=torch.cdouble)
    psi1n = torch.tensor([b0, ph1*b1], dtype=torch.cdouble)

    initial_states = Creating_states(coeff = coeff, abstract=False)     # Creating the two states with these coefficients
    psi0 = initial_states[0]
    psi1 = initial_states[1]    # created the states to be discriminated
    psi0p = [cm.polar(psi0[0])[0], cm.polar(psi0[1])[0]*np.exp(1j*(cm.polar(psi0[1])[1]-cm.polar(psi0[0])[1]))]
    psi1p = [cm.polar(psi1[0])[0], cm.polar(psi1[1])[0]*np.exp(1j*(cm.polar(psi1[1])[1]-cm.polar(psi1[0])[1]))]
    psi0t = torch.tensor(psi0p, dtype=torch.cdouble)
    psi1t = torch.tensor(psi1p, dtype=torch.cdouble)

    fid0 = torch.abs(torch.dot(psi0n, torch.conj(psi0t)))**2
    fid1 = torch.abs(torch.dot(psi1n, torch.conj(psi1t)))**2

    fid = [fid0.item(), fid1.item()]
    p = [p0.item(), p1.item()]

    return([fid,p])

In [ ]:
#coeff = [1,0,1/(2**.5),1/(2**.5)] 
#priors = [.5,.5]
coeff = [.9**.5,.1**.5,.1**.5,.9**.5]
priors = [.7,.3]
trials = [100*(i+1) for i in range(100)]
p0trials  = [priors[0] for i in trials] 
p1trials  = [priors[1] for i in trials]

lr0 = 0.01
#ip0 = [torch.tensor([val], requires_grad=True) for val in [torch.pi / 4, 0.0, 0.0, torch.pi / 2, 0.0]]
#print(ip0)
#ip = [torch.tensor([torch.pi / 4], requires_grad=True),torch.tensor([0.0], requires_grad=True),torch.tensor([0.0], requires_grad=True),torch.tensor([torch.pi / 2], requires_grad=True),torch.tensor([0.0], requires_grad=True)]

initial_states = Creating_states(abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]

output_f = []
output_p = []

pfid  = [1 for i in trials]    # perfect fidelity of 1
#for i,t in tqdm(enumerate(trials)):
    #print(f'i_N: {i}')
    #lr = update_lr(lr0,i)
    #ip = ip0
    #print(f'learning rate: {lr}')
    #output_f.append(grad_fid(lr,t,coeff,priors,POVM_elts)[0])
    #output_p.append(grad_fid(lr,t,coeff,priors,POVM_elts)[1])

output_f = [grad_fid(lr0, i, coeff, priors, POVM_elts)[0] for i in tqdm(trials)]
output_p = [grad_fid(lr0, i, coeff, priors, POVM_elts)[1] for i in tqdm(trials)]

outputf = list(map(list, zip(*output_f)))
outputp = list(map(list, zip(*output_p)))

#print(outputf)
#print(outputp)

In [ ]:
fig_a, (ax1a, ax2a) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1a.plot(trials, outputf[0], label=r'$F_0=|\langle \psi_0|\psi_0^{\,n}\rangle|^2$')
ax1a.plot(trials , pfid, "--", label=r'$F=1$')
ax1a.set_ylim(0.8,1)
ax1a.set_xlabel(' N (trials)')
ax1a.set_ylabel(r'Fidelity $F$')
#ax1a.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax1a.legend(loc='best')
ax2a.plot(trials, outputf[1], label=r'$F_1=|\langle \psi_1|\psi_1^{\,n}\rangle|^2$')
ax2a.plot(trials , pfid, "--", label=r'$F=1$')
ax2a.set_ylim(0.8,1)
ax2a.set_xlabel(' N (trials)')
ax2a.set_ylabel(r'Fidelity $F$')
#ax2a.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
ax2a.legend(loc='best')
plt.savefig('fid.pdf')

 
fig_c, (ax1c, ax2c) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1c.plot(trials, outputp[0], label=r'$p_{0}$ numerical')
ax1c.plot(trials, p0trials, '--', label=r'$p_{0}$ theoretical')
ax2c.plot(trials, outputp[1], label=r'$p_{1}$ numerical')
ax2c.plot(trials, p1trials, '--', label=r'$p_{1}$ theoretical')
ax1c.set_ylim(0,1)
ax2c.set_ylim(0,1)
ax1c.set_xlabel(' N (trials)')
ax2c.set_xlabel(' N (trials)')
ax1c.set_ylabel(r'Probability')
ax2c.set_ylabel(r'Probability')
ax1c.legend(loc='best')
ax2c.legend(loc='best')
#ax1c.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
#ax2c.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
plt.savefig('prior.pdf')

plt.show()

In [ ]:
coeff = [1,0,0,1]
priors = [.5,.5]
a = 0.7609137980205711
a = torch.tensor([a])
t0 = 5.776511204788547
t0 = torch.tensor([t0])
t1 = 2.595433051560899
t1 = torch.tensor([t1])
f0 = 0.5205233976049031
f0 = torch.tensor([f0])
f1 = 0.5264971946998136
f1 = torch.tensor([f1])

p0 = (torch.cos(a))**2
p1 = (torch.sin(a))**2
#print('p0+p1:', p0.item()+p1.item())
a0 = torch.cos(t0)
a1 = torch.sin(t0)
#print(f'a0^2 + a1^2: {(a0.item())**2 + (a1.item())**2}')
b0 = torch.cos(t1)
b1 = torch.sin(t1)
#print(f'b0^2 + b1^2: {(b0.item())**2 + (b1.item())**2}')
ph0 = torch.cos(f0) + 1j*torch.sin(f0)
ph1 = torch.cos(f1) + 1j*torch.sin(f1)
#print(f'p0: {p0.item()}, p1: {p1.item()}, a0: {a0.item()}, a1: {a1.item()}, b0: {b0.item()}, b1: {b1.item()}, ph0: {ph0.item()}, ph1: {ph1.item()}')

psi0n = torch.tensor([a0, ph0*a1], dtype=torch.cdouble)
psi1n = torch.tensor([b0, ph1*b1], dtype=torch.cdouble)
initial_states = Creating_states(coeff = coeff, abstract=False)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]    # created the states to be discriminated
psi0t = torch.tensor(psi0, dtype=torch.cdouble)
psi1t = torch.tensor(psi1, dtype=torch.cdouble)

fid0 = torch.abs(torch.dot(psi0n, torch.conj(psi0t)))**2
fid1 = torch.abs(torch.dot(psi1n, torch.conj(psi1t)))**2

fid = [fid0.item(), fid1.item()]
p = [p0.item(), p1.item()]

print(f'fid: {fid}, p: {p}')